In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [26]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%m/%d/%Y')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : dd/mm/YYYY
    """
    print("page ", page)
    if cat_link == 'news':
        url = "https://"+cat_link+".detik.com/indeks/all/"+str(page)+"?date="+date
    else :
        url = "https://"+cat_link+".detik.com/indeks/"+str(page)+"?date="+date
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.find('ul', attrs={'id':'indeks-container'})
    indeks = contentDiv.findAll('article')
    for post in indeks:
        link = [post.find('a', href=True)['href'], category]
        links.append(link)
        
    el_page = soup.find('div', class_="paging paging2")
    if el_page:
        max_page = int(soup.find('div', class_="paging paging2").findAll('a')[-2].text.replace('\n', '').strip(' '))
    
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

In [27]:
def getDetailBerita(links):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")
        
        #extract subcategory from breadcrumb
        bc = soup.find('div', class_="breadcrumb")
        if not bc:
            continue
        
        sub = bc.findAll('a')[1].text
        if ("foto" in sub.lower()) or  "video" in sub.lower(): 
            continue
            
        articles['subcategory'] = sub
        
        articles['id'] = int(soup.find("meta", attrs={'name':'articleid'})['content'])
        #category
        articles['category'] = link[1]
        articles['url'] = url
        
        article = soup.find('article')
        
        #extract date
        pubdate = soup.find("meta", attrs={'name':'publishdate'})['content']
        pubdate = pubdate.strip(' \t\n\r')
        articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%Y/%m/%d %H:%M:%S"), '%Y-%m-%d %H:%M:%S')
        
        #extract author
        articles['author'] = soup.find("meta", attrs={'name':'author'})['content']
        
        #extract title
        articles['title'] = article.find('div', class_="jdl").find('h1').text
        
        #source
        articles['source'] = 'detik'
        
        #extract comments count
        articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
        
        #extract tags
        tags = article.find('div', class_="detail_tag").findAll('a')
        articles['tags'] = ','.join([x.text for x in tags])
        
        #extract images
        articles['images'] = article.find('div', class_="pic_artikel").find('img')['src']
        
        #extract detail
        detail = article.find('div', class_="detail_text")
        
        #hapus link sisip
        for link in detail.findAll('table', class_="linksisip"):
            link.decompose()
        
        #hapus video sisip
        for tag in detail.findAll('div', class_="sisip_embed_sosmed"):
            tag.decompose()
            
        #hapus all setelah clear fix
        for det in detail.find('div', class_="clearfix mb20").findAllNext():
            det.decompose()
            
        #hapus all script
        for script in detail.findAll('script'):
            script.decompose()
        
        #extract content
        detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = re.sub(r'(Tonton juga).*','', content)
        print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [40]:
# links = getIndeksLink([], 1, 'news', 'news')

In [29]:
detail = getDetailBerita(links[10:13])

memasukkan berita id  4138819
memasukkan berita id  4138815
memasukkan berita id  4138816


In [31]:
detail

[{'author': 'Faiq Hidayat',
  'category': 'news',
  'comments': 0,
  'content': 'Jakarta -Gempa bumi berkekuatan 5,0 skala richter menguncang Lombok Utara, Nusa Tenggara Barat. Gempa itu tidak menimbulkan potensi tsunami.  Dilihat detikcom dari situs Badan Meteorologi, Klimatologi dan Geofisika (BMKG), Minggu (29/7/2018), gempa terjadi sekitar pukul 06.15 WIB.    Pusat gempa berada di laut tepatnya 37 km timur laut Lombok Utara. Kedalaman gempa mencapai 10 km.  Lokasi tepatnya berada di 8.16 LS - 116.54 BT. Belum diketahui ada atau tidaknya kerusakan akibat gempa ini.               (fai/haf)                                                            ',
  'id': 4138819,
  'images': 'https://akcdn.detik.net.id/community/media/visual/2015/07/10/ceaea5cf-a1ca-4de1-8dd0-96092a0fc333_169.jpg?w=780&q=90',
  'pubdate': '2018-07-29 06:33:50',
  'source': 'detik',
  'subcategory': 'Berita',
  'tags': 'gempa bumi,gempa',
  'title': 'Gempa 5,0 SR Guncang Lombok Utara',
  'url': 'https://news.detik

In [32]:
df = pd.DataFrame(detail)

In [33]:
df.to_csv('tes.csv')

In [34]:
df

,author,category,comments,content,id,images,pubdate,source,subcategory,tags,title,url
0,Faiq Hidayat,news,0,"Jakarta -Gempa bumi berkekuatan 5,0 skala rich...",4138819,https://akcdn.detik.net.id/community/media/vis...,2018-07-29 06:33:50,detik,Berita,"gempa bumi,gempa","Gempa 5,0 SR Guncang Lombok Utara",https://news.detik.com/berita/d-4138819/gempa-...
1,Haris Fadhil,news,0,Jakarta -Penyidik KPK Novel Baswedan kembali b...,4138815,https://akcdn.detik.net.id/community/media/vis...,2018-07-29 06:20:39,detik,Berita,"novel baswedan,kpk",Ini yang Dilakukan Novel Baswedan saat Kembali...,https://news.detik.com/berita/d-4138815/ini-ya...
2,Faiq Hidayat,news,0,Kecelakaan mobi di Tol Pejompongan (Foto: Dok...,4138816,https://akcdn.detik.net.id/community/media/vis...,2018-07-29 06:17:49,detik,Berita,"kecelakaan,kecelakaan mobil",Tiga Mobil Remuk Akibat Kecelakaan di Tol Pejo...,https://news.detik.com/berita/d-4138816/tiga-m...


In [4]:
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')  # Last I checked this was necessary.
options.add_argument('--disable-extensions')
# options.add_argument('--enable-javascript')

driver = webdriver.Chrome("../chromedriver.exe", chrome_options=options)

In [5]:
driver.get("https://nasional.kompas.com/read/2018/07/27/12344561/adiknya-ditangkap-kpk-zulkifli-mohon-maaf")
html = driver.page_source

In [151]:
soup = BeautifulSoup(html, 'html5lib')

In [152]:
soup.find('div', class_="span4 comments-count tright")

<div class="span4 comments-count tright" style="float:right;font-weight: bold;">Ada 43 komentar untuk artikel ini</div>

In [16]:
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, "html5lib")
el_page = soup.find('div', class_="paging paging2")
el_page

<div class="paging paging2">
                            <a href="https://news.detik.com/indeks/all/?date=07/28/2018">Â«</a>
            
            
                            <a href="https://news.detik.com/indeks/all/1?date=07/28/2018">1</a>
                            <a class="selected" href="https://news.detik.com/indeks/all/2?date=07/28/2018">2</a>
                            <a href="https://news.detik.com/indeks/all/3?date=07/28/2018">3</a>
                            <a href="https://news.detik.com/indeks/all/4?date=07/28/2018">4</a>
                            <a href="https://news.detik.com/indeks/all/5?date=07/28/2018">5</a>
            
                                                <a class="range">...</a>
                                <a href="https://news.detik.com/indeks/all/11?date=07/28/2018">11            
                            </a><a href="https://news.detik.com/indeks/all/3?date=07/28/2018">Â»</a>
                    </div>

In [41]:
detail.findAll('div', class_="sisip_embed_sosmed")

AttributeError: 'list' object has no attribute 'findAll'

In [10]:
url = "https://news.detik.com/berita/d-4138819/gempa-50-sr-guncang-lombok-utara"

In [11]:
response = requests.get(url)
html = response.text
# Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")

ConnectionError: HTTPSConnectionPool(host='news.detik.com', port=443): Max retries exceeded with url: /berita/d-4138819/gempa-50-sr-guncang-lombok-utara (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001A99472C5F8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [7]:
article = soup.find('article')
article

<article>
	<!-- s:judul -->
	<div class="jdl">
		<div class="date">Minggu 29 Juli 2018, 06:33 WIB</div>
		<h2></h2>
		<h1>Gempa 5,0 SR Guncang Lombok Utara</h1>
		<div class="author">Faiq Hidayat - detikNews</div>
	</div>
	<!-- e:judul -->
	<!-- s:share top -->

			
<script>
    var article = {
        idnews:4138819,
        idkanal:10    }
    var baseurl = "https://news.detik.com/"

    $(document).ready(function () {
        setTimeout(function(){
            shareBox.run($('.share-top-new'))}
        ,1000)

    });


</script>

<div class="share-top-new " id="share_top">
    <!--
    <a href="#" class="count">
        <strong>0</strong> SHARED
    </a>
-->
    <a class="fb" data-action="Artikel Biasa" data-category="Detail Artikel" data-desc="Gempa bumi berkekuatan 5,0 skala richter menguncang Lombok Utara, Nusa Tenggara Barat. Gempa itu tidak menimbulkan potensi tsunami." data-image="https://akcdn.detik.net.id/community/media/visual/2015/07/10/ceaea5cf-a1ca-4de1-8dd0-96092a0fc33

In [9]:
detail = article.find('div', class_="detail_text")
detail

<div class="detail_text" id="detikdetailtext">
			<!-- s:pic detail -->
			<!-- S:read image orientation if potrait load this -->
						<!-- E:read image orientation if potrait load this -->
			<!-- e:pic detail -->

			<b>Jakarta</b> -
							<a href="https://www.detik.com/search/searchall?query=gempa+bumi" target="_blank">Gempa bumi</a> berkekuatan 5,0 skala richter menguncang Lombok Utara, Nusa Tenggara Barat. Gempa itu tidak menimbulkan potensi tsunami.<br/><br/>Dilihat detikcom dari situs Badan Meteorologi, Klimatologi dan Geofisika (BMKG), Minggu (29/7/2018), gempa terjadi sekitar pukul 06.15 WIB.<br/><br/><table class="linksisip"><tbody><tr><td><div class="lihatjg"><strong>Baca juga: </strong><a data-action="Berita Pilihan" data-category="Detil Artikel" data-label="List Berita" href="https://news.detik.com/read/2018/07/28/224923/4138762/10/gempa-52-sr-guncang-maluku-barat-daya" target="_blank">Gempa 5,2 SR Guncang Maluku Barat Daya</a></div></td></tr></tbody></table><br/><br/><!

In [16]:
pubdate = soup.find("meta", attrs={'name':'publishdate'})['content']
pubdate = pubdate.strip(' \t\n\r')
pubdate
# articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%Y/%m/%d %H:%M:%S"), '%Y-%m-%d %H:%M:%S')

'2018/07/29 06:33:50'